<a href="https://colab.research.google.com/github/BrokLori/broklori.github.io/blob/master/Cloropleth_of_Overall_Child_Opportunity_Index_by_Census_Tract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## First "Map" of Buffalo

First step: Display a map of the Buffalo area. Areas of interest = Amherst (specifically Williamsville/Eggertsville/Snyder), Kenmore and Tonawanda. Add demographic data for these areas using census/ACS survey data (population, crime, diversity, school ratings etc.). 

**Desired outcome: User can select and/or rate preferred characteristics for a neighborhood, and based on that input, get some kind of visualization that demonstrates which neighborhoods would be the best fit.**

In [0]:
# Install libraries
!pip install geopandas
!pip install descartes

In [0]:
# Import libraries
# %matplotlib inline sets backend of matplotlib to 'inline' backend
# Output displayed below the cell
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes as dc

In [0]:
# Import gspread for Google Colab
!pip install --upgrade -q gspread

In [0]:
# Upload shapefiles from computer using files > upload tool

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
# Set filepath and load in a shapefile (Jupyter Notebook only)
# To get filepath, drag and drop in terminal (Jupyter Notebook only)

# Read in shapefile

fp = "tl_2018_36_tract.shp"
map_df = gpd.read_file(fp)

In [0]:
# Check data type
# This is not a normal dataframe, but a GEOdataframe
map_df.head()

In [0]:
# Filter COUNTYFP to only include Erie County (FIPS code 029)
map_df = map_df[map_df['COUNTYFP']=='029']

# Choose only columns that I want
map_df = map_df[['GEOID', 'geometry']]

# If you want to rename columns use the following commented code
# Map_df = map_df.rename(index=str, columns={"NAMELSAD": "Census Tract",
                                           #"geometry": "geometry"})
map_df.head()

In [0]:
# Preview map with no data in it
map_df.plot(figsize=(10,10))

In [0]:
# Import data from google sheets
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('Buffalo_Child_Opportunity').sheet1

# Get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows)
df.head()

In [0]:
# Remove the first row/header
# Create a new variable called 'new_header' from first row of dataset
new_header = df.iloc[0] 

# Slice the rest of your data, minus header row
df = df[1:] 

# Set the header row as the df header
df.columns = new_header

# Preview the 5 first rows of the new dataset
df.head()

In [0]:
# Select the columns that we want for the map
score = df[['FIPS', 'ALLCOMP']]

# Check dataframe
score.head()

In [0]:
# Rename columns to something simpler, if needed
# Make sure that merged columns match/not necessary if merging using GEOID
# Remove ', Erie County, New York' and replace with empty string
#score = df.replace(r'\, Erie County, New York.*', '', regex=True)
#score = score.rename(index=str, columns={"GEO.display-label": "Geo Label",
                                      #"HD01_VD01": "Estimate; Total"})

#score.head()

In [0]:
# Join the geodataframe with the cleaned up csv dataframe
# 'Outer' join to include values for Estimate;Total
merged = map_df.set_index('GEOID').join(score.set_index('FIPS'), how='outer')

# NaN values are taken as floats. If needed, fill with empty strings ''
#merged = merged.fillna('')
merged.head()

In [0]:
# Set a variable that will call the column we want to visualise on the map
variable = 'ALLCOMP'

# Set the range for the choropleth
vmin, vmax = -2,2

# Create figure and axes for Matplotlib
# Missing ) from source code
fig, ax = plt.subplots(1, figsize=(10, 10))

# Create map
merged.plot(column=variable, cmap='Blues', linewidth=0.8, ax=ax, edgecolor='0.8')

# Customize and add annotations

# Remove the axis
ax.axis('off')

# Add a title
ax.set_title('Overall Child Opportunity Index Score by Census Tract: \nErie County, NY', \
              fontdict={'fontsize': '16',
                        'fontweight' : '3'})

# Create an annotation for the  data source
ax.annotate('Source: diversitydatakids.org',
           xy=(0.1, .08), xycoords='figure fraction',
           horizontalalignment='left', verticalalignment='top',
           fontsize=10, color='#555555')

# Create colorbar as a legend
sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
cbar = fig.colorbar(sm)

# This will save the figure as a high-res png. you can also save as svg
fig.savefig('Overall Child Opportunity Index.png', dpi=300)